### MY470 Computer Programming

### Week 4 Assignment, MT 2022

#### \*\*\* Due 12:00 noon on Monday, October 24 \*\*\*

---
### Writing your own k-means clustering algorithm

K-means clustering is a simple unsupervised machine-learning method for cluster analysis. The aim of the method is to partition a set of points into k clusters, such that each point is assigned to the nearest cluster. The algorithm iterates through two steps:

1. Assign each data point to the cluster with the nearest centroid
2. Update the centroids of the clusters given the new assignment

The algorithm converges when the assignments no longer change. Since the intial assignment to clusters is largely random, there is no guarantee that the optimum assignment is found. So it is common to run the algorithm multiple times and use different starting conditions.

In this assignment, we will implement a much simplified version of the k-means clustering algorithm. Rather than running the algorithm until convergence, we will repeat the above two steps a large but fixed number of times. In addition, we will initialize only once, using a naive method according to which we randomly choose k points from the data to use as initial cluster centroids. 

(In real life, you will of course use a library to implement such an algorithm. In Python, you can do this using [scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).)

For the assignment, we will additionally use data from the file "Wholesale customers data.csv". The file contains information on the annual spending on diverse product categories for the clients of a wholesale distributor. The data are obtained from the [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/index.php) and you can find more information about them [here](http://archive.ics.uci.edu/ml/datasets/Wholesale+customers#).

#### Hints

Use docstrings to describe your functions. We will subtract points from your mark if you do not use appropriate description of your code.

In [1]:
# We will first import the modules we need
# Edit this cell if you prefer to use alternative modules or libraries

# You will need the math module to estimate the square root.
# To get the square root of num, use math.sqrt(num)
import math
import csv
import random

### Problem 1: Function to estimate Euclidean distance between two points

Write a function called `get_distance` that calculates the Euclidean distance between two n-dimensional points. The function should take two lists as arguments, where each list contains the n coordinates of each of the two points. 

Test your function for the points [0, 3, 0] and [4, 0, 0].

#### Hints

You can read about the definition of Euclidean distance on [Wikipedia](https://en.wikipedia.org/wiki/Euclidean_distance).


In [2]:
# Enter your answer to Problem 1 below. 
def get_distance(x,y):
    """Assumes x and y are the coordinates of n-dimensional points.
    Returns the Euclidean distance between the two points.
    """
    s = 0
    for i in range(len(x)):
        s = s + (x[i] - y[i])**2
    return(math.sqrt(s))

x = [0, 3, 0]
y = [4, 0, 0]

get_distance(x,y)

5.0

### Problem 2: Function to estimate the centroid of a collection of points

Write a function called `get_centroid` that estimates the centroid of a collection of n-dimensional points. The function should take one list as an argument, which contains each of the points entered as a list of n coordinates. The function should return a list with the coordinates of the virtual center point.

Test your function for the points in `test_lst` entered below.

#### Hints

The coordinate of the centroid in each dimension is the mean of the coordinates of all the points in that dimension.


In [3]:
# Source: Python documentation, zip() function, https://docs.python.org/3/library/functions.html#zip
test_lst = [[0,0,0], [0,0,1], [0,1,0], [1,0,0], 
            [0,1,1], [1,0,1], [1,1,0], [1,1,1]]

# Enter your answer to Problem 2 below. 
def get_centroid(lst):
    """Assumes that the input is a list of coordinates of several
    n-dimensional points. Returns the coordinates for the centroid
    of these points.
    """
    sum_coord = len(lst[0]) * [0]
    coord = []
    for elem in lst:
        sum_coord = [sum(i) for i in zip(sum_coord,elem)]
    for i in sum_coord:
        coord.append(i/len(test_lst))
    return(coord)

get_centroid(test_lst)

[0.5, 0.5, 0.5]

---
### Problem 3: Function to read data

Write a function called `get_data` that opens the file "Wholesale customers data.csv" and returns all the data in a list. Each element in the list should be a list of each customer's annual spending on fresh products, milk products, grocery products, frozen products, detergents and paper products, and delicatessen products. In other words, your list should contain 440 elements (customers), each of which contains six numeric elements (amounts spent on products). The function does not need to take any arguments.

Test your function by saving the data it returns in a variable called `data`. Then print the first two elements of `data`.


#### Hints

Use the csv module to read the file. You can read how to do this [here](https://docs.python.org/3/library/csv.html). Make sure you do not include the column names in the data. 

In [4]:
# Source: Python documentation, csv module, https://docs.python.org/3/library/csv.html
# Enter your answer to Problem 3 here. 
def get_data():
    """Opens the file "Wholesale customers data.csv".
    Returns a list every element of which is a list of each customer's
    spending on fresh products, milk products, grocery products, 
    frozen products, detergents and paper products, and delicatessen products respectively.
    """
    data = []
    with open("Wholesale customers data.csv", newline='') as csvfile:
        data_reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        data = []
        for row in data_reader:
            customer_list = []
            split_string = row[0].split(',')
            if split_string[0].isnumeric() == True:
                for i in range(2,len(split_string)):
                    customer_list.append(float(split_string[i]))
                data.append(customer_list)
    return(data)

data = get_data()
print(data[:2])

[[12669.0, 9656.0, 7561.0, 214.0, 2674.0, 1338.0], [7057.0, 9810.0, 9568.0, 1762.0, 3293.0, 1776.0]]


---
### Problem 4: Function to implement k-means algorithm

Write a function called `kmeans` that clusters a collection of points into k clusters using a simplified version of the k-means algorithm. The function should take two arguments: 

1. `points` – a list of n-dimensional points, and
2. `k` – an integer that defines the number of desired clusters. 

The function should return two things: 

1. A clustering – a list of `k` clusters, each of which is a list of points (each of which is a list of coordinates)
2. A list of the centroids for each of the `k` clusters. Each centroid is essentially a point, so it should be presented as a list of coordinates.

Write your code around the detailed comments and the helping code below.

Test your function on the data from Problem 3 for k = 3. For each of the three clusters, print the number of customers assigned to it and the cordinates of its centroid.


In [5]:
# Enter your answer to Problem 4 in-between the code and comments below. 

def kmeans(points, k):
    """Assumes points is a list of coordinates for n-dimensional datapoints, 
    k is an integer standing for the number of clusters. 
    Returns clusters and their centroids after performing the k-means clustering algorithm.
    """
    
    # Select k random points to use as initial centroids
    init = random.sample(points, k)

    # Create a list of k lists to contain the points assigned to each cluster.  
    clusters = [[] for i in init]
    
    # Create a list to keep the centroids of the k clusters. 
    # For now, this list will contain the points from init.
    centroids = [i for i in init]
    
    # You now need to assign each point to the cluster 
    # with the closest centroid. Use the get_distance function 
    # you wrote in Problem 1 for this.
    for i in points:
        distances_list = [get_distance(i,j) for j in centroids]
        min_index = distances_list.index(min(distances_list)) 
        clusters[min_index].append(i)
    
    # You should then update the variable "clusters" to be 
    # the new clustering and update the variable "centroids" 
    # to contain the centroids of the clusters in this new clustering.
    # Use the function you wrote in Problem 2 to estimate the centroids.
    centroids = [get_centroid(i) for i in clusters]
    
    # Repeat the process described above for 100 iterations. 
    # The idea is that each new repetition refines the clustering 
    # because it starts from the centroids of the previous clustering. 
    # If we repeat the process long enough, the assignment to 
    # clusters and the centroids will become stable.
    for i in range(100):
        for cl in clusters:
            current_cluster_index = clusters.index(cl)
            for elem in cl:
                distances_list = [get_distance(elem,j) for j in centroids]
                min_index = distances_list.index(min(distances_list)) 
                if elem not in clusters[min_index]:
                    clusters[min_index].append(elem)
                    clusters[current_cluster_index].remove(elem)
        centroids = [get_centroid(i) for i in clusters]
    
    return(clusters,centroids)

clusters, centroids = kmeans(data,3)
for cl in clusters:
    cluster_index = clusters.index(cl)
    print(len(cl), centroids[cluster_index])

125 [190683.75, 74801.625, 107571.875, 46491.125, 39090.875, 19014.875]
252 [364303.25, 197783.375, 264409.625, 99441.375, 97639.25, 52776.25]
63 [105029.375, 46209.625, 65338.75, 23023.75, 21752.0, 12076.75]


---

### Evaluation

| Problem | Mark     | Comment   
|:-------:|:--------:|:----------------------
| 1       |   /2    |              
| 2       |   /2    | 
| 3       |   /2    | 
| 4       |   /6    | 
| Legibility      |   /2    | 
| Modularity      |   /2    | 
| Efficiency      |   /4    | 
|**Total**|**/20**  | 